In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/home/ektov1-av_ca-sbrf-ru/bin/python35
import os
import sys
curruser = os.environ.get('USER')

# sys.path.insert(0, '/opt/workspace/{user}/system/support_library/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/libs/'.format(user=curruser))
# sys.path.insert(0, '/opt/workspace/{user}/system/labdata/lib/'.format(user=curruser))


# sys.path.insert(0, './../src')
sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

#import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(filename='./__tera_fload__.log',level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
import decimal
import datetime
from getpass import getpass

import jaydebeapi

from transliterate import translit

from spark_connector import SparkConnector
from sparkdb_loader import spark
from connector import OracleDB, TeraDB
import pyspark
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.utils import AnalysisException

from pyspark.sql import Row

import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
import pandas as pd
import csv
from tqdm import tqdm

# sing = tendo.singleton.SingleInstance()

# os.chdir('/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/Clickstream_Analytics/AutoUpdate/')
# os.chdir('/opt/workspace/{}/notebooks/clickstream/AutoUpdate/'.format(curruser))


##-----------------------------------------
conn_schema = 'sbx_team_digitcamp' #'dl_t_team_ds_kb_sme' 'dl_t_team_speech_analytics'
new_schema = 'dl_team_digitcamp'
ga_schema = 'sklod_external_google_analytics'
##-----------------------------------------


def log(message, logger,
        print_log : bool = True):
    if not print_log:
        return

    logger.info(message)
    print(message, file=sys.stdout)
    # print(message, file=sys.stderr)

def show(self, n=10):
    return self.limit(n).toPandas()

def typed_udf(return_type):
    '''Make a UDF decorator with the given return type'''

    def _typed_udf_wrapper(func):
        return f.udf(func,return_type)

def essense(channel: str, prod_cd: str):
    message = "{}: {} retargeting".format(channel, prod_cd)
    return message

essense_udf = f.udf(essense, StringType())

pyspark.sql.dataframe.DataFrame.show = show

def print_and_log(message: str):
    print(message)
    logger.info(message)
    return None


In [4]:
def block_iterator(iterator,size):
    """
    dest:
        сервисная функция для итерации по блокам данных внутри RDD. 
        Чем больше size при увеличении количества потоков, тем быстрее обработка
    args:
        iterator-объект 
        size - размер элементов для единичной итерации
    return:
        вычисляемый объект bucket
 
    """
    bucket = list()
    for e in iterator:
        bucket.append(e)
        if len(bucket) >= size:
            yield bucket
            bucket = list()
    if bucket:
        yield bucket

def block_classify(iterator):
    import os
    import sys
    import pandas as pd
    import json

    for out in block_iterator(iterator, 100):   

        cols = [col for col,_ in col_bc.value]
        currschema = StructType([StructField(col, typesmap_rdd_bc.value[col]) for col in cols])
        sdf_proc = sp.sql.createDataFrame(out.collect(), schema=currschema)
        break
    
    return sdf_proc

def collectRowsByIndex(i, it, indxs):
    out = []
    if i in indxs:
         out.extend(list(it)) #islice(it,0,5) 
    else:
        pass

    return out 

In [5]:
print("### Starting spark context. Run!")

sp = spark(schema=conn_schema,
           dynamic_alloc=False,
           kerberos_auth=False,
           numofinstances=8, 
           numofcores=8,
           executor_memory='30g', 
           driver_memory='30g'
           )
hive = sp.sql

print(sp.sc.version)

### Starting spark context. Run!
2.4.0.cloudera2


====================================================== 2022-03-30 ======================================================
# __init__ : begin


## Set a Conection to Teradata

In [7]:
TERADATA_HOST = "TDSB15"
DB = ""
USERNAME = ""
PASSWORD = getpass()
db = TeraDB(TERADATA_HOST, DB, USERNAME, PASSWORD)

········


## Get actual partitions from MetaStore

In [6]:
from itertools import cycle

In [7]:
conn_schema = 'sbx_t_team_cvm' #'sbx_team_digitcamp' #'sbx_t_team_cvm'
table_name = 'lal_db_hist_out' #'ma_cmdm_ma_deal_new' #'lal_db_hist_in'

In [ ]:
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
descr = hive.sql("describe extended {}.{}".format(conn_schema,table_name)).collect()
push_down = True
loop_rows = cycle(descr)
nextelem = next(loop_rows)
while push_down:
    thiselemen, nextelem = nextelem, next(loop_rows)
    if nextelem.asDict()['col_name'] =='# Partition Information':
        next(loop_rows)
        part_col = next(loop_rows)
        part_col_info = part_col.asDict()
        push_down = False
        
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [8]:
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
part_name = part_col_info['col_name']
# set the fix number of sorted patitions to catch from metastore
numdays = 15       
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

hasPartitioned = len([item.asDict() for item in descr if item.asDict()['col_name'] =='# Partition Information']) > 0
if hasPartitioned:
    try:
        parts = hive.sql("show partitions {}.{}".format(conn_schema,table_name)).collect()
        parts = [part['partition'] for part in parts if not part['partition'].endswith('__HIVE_DEFAULT_PARTITION__')]
        parts = sorted(parts,reverse=True)
        max_part = parts[0]
        extract_date=re.compile("\d{4}\-\d{2}\-\d{2}")
        ext = extract_date.search(max_part)
        try:
            max_trunc_dt = ext.group(0)
        except:
            max_trunc_dt = None
        if part_col_info['data_type'] in ('date', 'timestamp', 'string') and (max_trunc_dt is not None): 
            parts = sorted(parts, reverse=True, key=lambda x: 
                           datetime.datetime.strptime(x.split(part_name+'=')[-1].split('/')[0], '%Y-%m-%d'))
        else:
            parts = sorted(parts, reverse=True, key=lambda x: int(x.split(part_name+'=')[-1].split('/')[0]))
        last_part_lst = sorted(list(set([part.split(part_name+'=')[-1].split('/')[0] for part in parts])), 
                               reverse=True)[:numdays]
    except (AnalysisException, IndexError):
        last_part_lst = None
else:
    last_part_lst = None
#/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [9]:
part_name

'CREATE_DT_DAY'

In [10]:
last_part_lst

['2022-03-17',
 '2022-03-16',
 '2022-03-03',
 '2022-03-01',
 '2022-02-01',
 '2022-01-31',
 '2021-12-24',
 '2021-12-23',
 '2021-12-06',
 '2021-11-29',
 '2021-11-19',
 '2021-11-18',
 '2021-11-15',
 '2021-10-18',
 '2021-10-05']

## Filter Corrupted Partitions

In [11]:
HDFS_PATH = "hdfs://clsklsbx/user/team/{}/hive/{}/{}".format(conn_schema.split("sbx_")[-1].split("t_")[-1], 
                                                             table_name, 
                                                             part_name)+"={}/"
noncorrupCTL = []
for ctl in last_part_lst:
    try:
        sdf = hive.read.load(path=HDFS_PATH.format(ctl), format='parquet').limit(10)
        noncorrupCTL.append(ctl)
    except (AnalysisException) as err:
        if "Unable to infer schema for Parquet" in str(err):
            print(" Unable to infer schema for Parquet. It must be specified manually")
            
noncorrupCTL

['2022-03-17',
 '2022-03-16',
 '2022-03-03',
 '2022-03-01',
 '2022-02-01',
 '2022-01-31',
 '2021-12-24',
 '2021-12-23',
 '2021-12-06',
 '2021-11-29',
 '2021-11-19',
 '2021-11-18',
 '2021-11-15',
 '2021-10-18',
 '2021-10-05']

## Select Appropriate Rows Under User Conditions

In [12]:
sdf = hive.sql('''select * from {}.{} 
                  where {} between '{}' and '{}' '''.format(conn_schema,
                                                            table_name,
                                                            part_name.lower(),
                                                            noncorrupCTL[-1],
                                                            noncorrupCTL[0]
                                                           )
              )

In [13]:
sdf = sdf.withColumn("row_id", f.monotonically_increasing_id().cast(IntegerType()))

In [14]:
sdf.cache()

DataFrame[INN: string, MODEL_NAME: string, HYPOTHESIS: string, CLIENT_TYPE: string, ID_SCEN: string, CREATE_DT: timestamp, ONLY_SB: decimal(20,0), DEAL_EVENT_DT: timestamp, MODEL_TYPE: string, TARGET: decimal(20,0), CREATE_DT_DAY: date, row_id: int]

In [129]:
sdf.count()

1477549

In [132]:
sdf.show()

,INN,MODEL_NAME,HYPOTHESIS,CLIENT_TYPE,ID_SCEN,CREATE_DT,ONLY_SB,DEAL_EVENT_DT,MODEL_TYPE,TARGET,CREATE_DT_DAY,row_id
0,592012195609,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,0
1,592012237030,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,1
2,592012345117,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,2
3,592012369679,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,3
4,592012376965,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,4
5,592012426180,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,5
6,592012851219,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,6
7,592012888843,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,7
8,592013018634,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,8
9,592013084281,1-34DIS7P2,validation,all,test45,2022-01-31 11:42:11.301388,0,2022-01-31 11:42:11.301388,lal,1,2022-01-31,9


## Map ColumnTypes from Spark to Teradata

In [14]:
df = sdf.limit(1000).toPandas()

In [ ]:
#**************************************************************************************************************
#**************************************************************************************************************
## Place Row_ID on the first column-position in your database. 
## Teradata always use the first column as Primary Index if not explicitly specified in 
## `INSERT INTO SELECT * FROM ` clause
#**************************************************************************************************************
#**************************************************************************************************************

In [25]:
cols_specific = {}
cols_specific["ROW_ID"] = ['INTEGER', 'INT']
#cols_specific {COL_NAME: [TERATYPE, HIVE_TYPE]}

isDateFmt = False
PRIMARY_INDX = 'ROW_ID'

str_ = \
'''
CREATE MULTISET TABLE {0}, 
    NO FALLBACK,
    NO BEFORE JOURNAL, 
    NO AFTER JOURNAL,
    CHECKSUM = DEFAULT,
    DEFAULT MERGEBLOCKRATIO,
    MAP = TD_MAP2
    (
        ROW_ID INTEGER,
'''
for column_name, column in df.iteritems():
    try:
        if isinstance(column[column.first_valid_index()], str):
            if (df[column_name].str.len().max() >= 4000) or ('COMMONSEGMENTOUID' in column_name.upper()):
    #             df.drop(columns=[column_name], inplace=True)
                str_+=column_name.upper() + ' ' +'CLOB, '
                cols_specific[column_name] = ['CLOB', 'STRING']
            else:
                if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                    str_+="\t" + column_name.upper() + ' ' +'VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,\n'
                    cols_specific[column_name] = ['VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC', 'STRING']
                elif part_name.upper() in column_name.upper():
                    str_+="\t" + column_name.upper() + ' ' +"DATE FORMAT 'YYYY-MM-DD',\n"
                    cols_specific[column_name] = ["DATE FORMAT 'YYYY-MM-DD'", 'DATE_FORMAT({}, "yyyy-MM-dd")']
                else:
                    str_+="\t" + column_name.upper() + ' ' +'VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\n'
                    cols_specific[column_name] = ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC', 'STRING']
        elif isinstance(column[column.first_valid_index()], np.integer) and (column_name.upper() != "ROW_ID"):
            if len(str(column[column.first_valid_index()])) < 1.e6:
                str_+="\t" + column_name.upper() + ' ' +'INTEGER,\n'
                cols_specific[column_name] =['INTEGER', 'INT']
            else:
                str_+="\t" + column_name.upper() + ' ' +'INTEGER, '
                cols_specific[column_name] =['INTEGER', 'BIGINT']            
        elif (
               ( isinstance(column[column.first_valid_index()], decimal.Decimal) ) or 
               ( isinstance(column[column.first_valid_index()], float) )
             ):
    #         df[column_name] = df[column_name].fillna(0.0)
            str_+="\t" + column_name.upper() + ' ' +'FLOAT,\n' 
            cols_specific[column_name] = ['FLOAT', 'FLOAT']
        elif (
                isinstance(column[column.first_valid_index()], pd.Timestamp) or 
                isinstance(column[column.first_valid_index()], datetime.date)
             ):
            if hasattr(column[column.first_valid_index()],'minute'):
                if column[column.first_valid_index()].minute == 0:
                    isDateFmt = True
                elif column[column.first_valid_index()].microsecond != 0:
                    isDateFmt = False
                    str_+="\t" + column_name.upper() + ' ' +"TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS.S(6)',\n"
                    cols_specific[column_name] = ["TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS.S(6)'", 
                                                  'DATE_FORMAT({}, "yyyy-MM-dd HH:mm:ss.SSSSSS")']                    
                else:
                    isDateFmt = False
                    str_+="\t" + column_name.upper() + ' ' +"TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS',\n"
                    cols_specific[column_name] = ["TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS'", 
                                                  'DATE_FORMAT({}, "yyyy-MM-dd HH:mm:ss")']
            else:
                isDateFmt = True
            if isDateFmt:
                str_+="\t" + column_name.upper() + ' ' +"DATE FORMAT 'YYYY-MM-DD',\n"
                cols_specific[column_name] = ["DATE FORMAT 'YYYY-MM-DD'", 'DATE_FORMAT({}, "yyyy-MM-dd")']            
        else:
            None  
    except:
        str_+=column_name.upper() + ' ' +'VARCHAR(800),\n'
        cols_specific[column_name] = ['VARCHAR(800)', 'STRING']
        
res=str_.strip()[:-1] + '\n\t)' + '\nPRIMARY INDEX ({});'.format(PRIMARY_INDX)

cr_tbl_sql = res

print(cr_tbl_sql)

CREATE MULTISET TABLE {0}, 
    NO FALLBACK,
    NO BEFORE JOURNAL, 
    NO AFTER JOURNAL,
    CHECKSUM = DEFAULT,
    DEFAULT MERGEBLOCKRATIO,
    MAP = TD_MAP2
    (
        ROW_ID INTEGER,
	INN VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,
	MODEL_NAME VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	HYPOTHESIS VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	CLIENT_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	ID_SCEN VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	CREATE_DT TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS',
	ONLY_SB FLOAT,
	DEAL_EVENT_DT TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS',
	MODEL_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	TARGET FLOAT,
	CREATE_DT_DAY DATE FORMAT 'YYYY-MM-DD'
	)
PRIMARY INDEX (ROW_ID);


## Add Partitioning Instructions

In [26]:
PART_COL_NAME = part_name
cr_part_tbl_query = cr_tbl_sql[:-1] + \
"\nPARTITION BY RANGE_N({} BETWEEN DATE '2020-01-01' AND DATE '2025-01-01' EACH INTERVAL '1' DAY);".format(PART_COL_NAME)

In [27]:
print(cr_part_tbl_query)

CREATE MULTISET TABLE {0}, 
    NO FALLBACK,
    NO BEFORE JOURNAL, 
    NO AFTER JOURNAL,
    CHECKSUM = DEFAULT,
    DEFAULT MERGEBLOCKRATIO,
    MAP = TD_MAP2
    (
        ROW_ID INTEGER,
	INN VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,
	MODEL_NAME VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	HYPOTHESIS VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	CLIENT_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	ID_SCEN VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	CREATE_DT TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS',
	ONLY_SB FLOAT,
	DEAL_EVENT_DT TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS',
	MODEL_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,
	TARGET FLOAT,
	CREATE_DT_DAY DATE FORMAT 'YYYY-MM-DD'
	)
PRIMARY INDEX (ROW_ID)
PARTITION BY RANGE_N(CREATE_DT_DAY BETWEEN DATE '2020-01-01' AND DATE '2025-01-01' EACH INTERVAL '1' DAY);


In [28]:
cols_specific

{'CLIENT_TYPE': ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC',
  'STRING'],
 'CREATE_DT': ["TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS'",
  'DATE_FORMAT({}, "yyyy-MM-dd HH:mm:ss")'],
 'CREATE_DT_DAY': ["DATE FORMAT 'YYYY-MM-DD'",
  'DATE_FORMAT({}, "yyyy-MM-dd")'],
 'DEAL_EVENT_DT': ["TIMESTAMP FORMAT 'YYYY-MM-DDBHH:MI:SS'",
  'DATE_FORMAT({}, "yyyy-MM-dd HH:mm:ss")'],
 'HYPOTHESIS': ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC',
  'STRING'],
 'ID_SCEN': ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC', 'STRING'],
 'INN': ['VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC', 'STRING'],
 'MODEL_NAME': ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC',
  'STRING'],
 'MODEL_TYPE': ['VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC',
  'STRING'],
 'ONLY_SB': ['FLOAT', 'FLOAT'],
 'ROW_ID': ['INTEGER', 'INT'],
 'TARGET': ['FLOAT', 'FLOAT']}

## Batch Exporting to Local storage of Hive tbl using RDD MapPartitions with Index

In [30]:
COLS_ORDER_LST__ = ["ROW_ID"] + [col.upper() for col in sdf.columns if col not in ("row_id")]
COLS_ORDER_LST__

['ROW_ID',
 'INN',
 'MODEL_NAME',
 'HYPOTHESIS',
 'CLIENT_TYPE',
 'ID_SCEN',
 'CREATE_DT',
 'ONLY_SB',
 'DEAL_EVENT_DT',
 'MODEL_TYPE',
 'TARGET',
 'CREATE_DT_DAY']

In [103]:
BATCH_SPLIT__ = 2
numPartitions = sdf.rdd.getNumPartitions()

batches = np.array_split(np.arange(0, numPartitions), BATCH_SPLIT__)
tuple_batches = [(i, batches[i]) for i in range(len(batches))]

name_postfix , partitions = tuple_batches[0]

In [104]:
typesmap_rdd={}

for column_name, column in sdf.dtypes:
    if column == 'string':
        typesmap_rdd[column_name] = StringType()
    elif 'decimal' in column:
        digits = int('{}'.format(column.split('(')[1].split(',')[0]))
        prec   = int('{}'.format(column.split('(')[1].split(',')[1][:-1]))
        typesmap_rdd[column_name] = DecimalType(digits,prec)
    elif column == 'double':
        typesmap_rdd[column_name] = DoubleType()        
    elif column == 'float':
        typesmap_rdd[column_name] = FloatType()
    elif column == 'int':
        typesmap_rdd[column_name] = IntegerType()       
    elif column == 'bigint':
        typesmap_rdd[column_name] = LongType()
    elif column == 'timestamp':
        typesmap_rdd[column_name] = TimestampType()
    elif column == 'date':
        typesmap_rdd[column_name] = DateType()
        
typesmap_rdd

{'CLIENT_TYPE': StringType,
 'CREATE_DT': TimestampType,
 'CREATE_DT_DAY': DateType,
 'DEAL_EVENT_DT': TimestampType,
 'HYPOTHESIS': StringType,
 'ID_SCEN': StringType,
 'INN': StringType,
 'MODEL_NAME': StringType,
 'MODEL_TYPE': StringType,
 'ONLY_SB': DecimalType(20,0),
 'TARGET': DecimalType(20,0),
 'row_id': IntegerType}

In [102]:
tmp_tbl_name = "tmp_hive2tera_cast"
EXPORT_PATH = "/opt/workspace/{curruser}/notebooks/TERA_FAST_LOAD/csv".format(curruser = os.environ.get('USER'))
EXPORT_PATH

'/opt/workspace/ektov1-av_ca-sbrf-ru/notebooks/TERA_FAST_LOAD/csv'

In [105]:
%%time
cols = [col for col,_ in sdf.dtypes]
for i, partlst in tqdm_notebook(tuple_batches[:]):
        
    print("# Step {}; part length: {}".format(i, len(partlst)))
    res = sdf.rdd.mapPartitionsWithIndex(lambda i,it: collectRowsByIndex(i,it,indxs=partlst))
    
    currschema = StructType([StructField(col, typesmap_rdd[col]) for col in cols])
    sdf_proc = hive.createDataFrame(res, schema=currschema)
    
    sdf_proc.registerTempTable(tmp_tbl_name)

    _str = '''SELECT '''
    for col_name, spec in cols_specific.items(): 
        if '{' not in spec[1]:
            _str+='''CAST({col} AS {type}) {col}, '''.format(col=col_name, type=spec[1])
        else:
            _str+='''CAST({} as {}) {}, '''.format(spec[1].format(col_name), 
                                                   spec[0].split("FORMAT")[0].split(" ")[0].strip(), col_name)
    fin_query = _str.strip()[:-1] + " FROM {}".format(tmp_tbl_name)

    sdf_casted = hive.sql(fin_query)
    sdf_casted = sdf_casted.select( ["row_id"]+[col for col in sdf.columns if col not in ("row_id")])    
    
    df = sdf_casted.toPandas()
    df.to_csv(os.path.join(EXPORT_PATH, "{}_{}.csv".format(table_name, i)), sep=",", encoding="utf-8", index=False)

# Step 0; part length: 9
# Step 1; part length: 8

CPU times: user 4min 50s, sys: 14.5 s, total: 5min 5s
Wall time: 5min 57s


## GENERATE Config for FastLoad utility

In [31]:
config_str=''

config_str+=\
'''
logon {HOST}/{user}, {_pass};

set record vartext  ",";
record 2;

database {db};
drop table {db}.{tbl}_err;
drop table {db}.{tbl}_err1;
drop table {db}.{tbl}_err2;
drop table {db}.{tbl};

'''.format(HOST = TERADATA_HOST,
           user = USERNAME,
           _pass = PASSWORD,
           db   = DB,
           tbl  = table_name
          )

config_str+=cr_part_tbl_query.format(table_name)

config_str+=\
'''
create error table {db}.{tbl}_err for {db}.{tbl};
begin loading {db}.{tbl}
    errorfiles {db}.{tbl}_err1, {db}.{tbl}_err2
    checkpoint 1500000;

define

'''.format(
           db   = DB,
           tbl  = table_name
          )

in_str=""
for col in COLS_ORDER_LST__:
    in_str+="in_{} (VARCHAR(500)),\n".format(col)
in_str=in_str.strip()[:-1]

config_str+=in_str

config_str+=\
'''

file="U:\FAST_LOAD\csv\lal_db_hist_in_0.csv";

SHOW;

insert into {db}.{tbl} (

'''.format(db=DB,
           tbl=table_name
          )

in_str=""
for col in COLS_ORDER_LST__:
    in_str+="{},\n".format(col)
in_str=in_str.strip()[:-1]

config_str+=in_str

config_str+=\
'''

)

values (

'''
 
in_str=""
for col in COLS_ORDER_LST__:
    _type = cols_specific[col]
    if 'FORMAT' in _type[0]:
        fmt = _type[0].split("FORMAT")[-1].strip()
        in_str+=":in_{}(FORMAT {}),\n".format(col, fmt)
    else:
        in_str+=":in_{},\n".format(col)
in_str=in_str.strip()[:-1]

config_str+=in_str    

config_str+=\
'''

);

end loading;
logoff;
'''    

## Write CONFIG into file

In [32]:
CONF_OUT_PATH__ = "/opt/workspace/{curruser}/notebooks/TERA_FAST_LOAD/run.fld".format(curruser = os.environ.get('USER'))

with open(CONF_OUT_PATH__, "w", encoding="utf8") as fout:
#     for ind, line in enumerate(config_str.split("\n")):
    fout.writelines(config_str)

In [ ]:
#######################################################################################################
#######################################################################################################
## Copy the content of current `/opt/workspace/../TERA_FAST_LOAD` folder into VARM disk `U:\FAST_LOAD\`
## CHDIR -D U:\FAST_LOAD\
## run teradata loader via executing command: `fastload -c utf8 <run.fld> log.txt`
#######################################################################################################
#######################################################################################################

## Check the state of the tbl being exported via FastLoad utility

In [12]:
JDBC_ARGUMENTS = "LOGMECH=LDAP, CHARSET=UTF8, TYPE=FASTEXPORT, COLUMN_NAME=ON, MAYBENULL=ON"
#DB = 'PRD_DB_CLIENT4D_DEV1'
conn = jaydebeapi.connect(
    jclassname="com.teradata.jdbc.TeraDriver",
    url="jdbc:teradata://{}/database={} , {}".format(TERADATA_HOST, DB, JDBC_ARGUMENTS),
    driver_args={"user": USERNAME, "password": PASSWORD},
    jars=['/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/tdgssconfig.jar',
          '/home/ektov1-av_ca-sbrf-ru/notebooks/drivers/terajdbc4.jar']
)
curs = conn.cursor()

In [13]:
curs.execute("SHOW TABLE PRD_DB_CLIENT4D_DEV1.{};".format(table_name))
curs.fetchall()

[("CREATE MULTISET TABLE PRD_DB_CLIENT4D_DEV1.lal_db_hist_in ,NO FALLBACK ,\r     NO BEFORE JOURNAL,\r     NO AFTER JOURNAL,\r     CHECKSUM = DEFAULT,\r     DEFAULT MERGEBLOCKRATIO,\r     MAP = TD_MAP2\r     (\r      ROW_ID INTEGER,\r      INN VARCHAR(20) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      MODEL_NAME VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      HYPOTHESIS VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      CLIENT_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      ID_SCEN VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      CREATE_DT TIMESTAMP(6) FORMAT 'YYYY-MM-DDBHH:MI:SS',\r      ONLY_SB FLOAT,\r      DEAL_EVENT_DT TIMESTAMP(6) FORMAT 'YYYY-MM-DDBHH:MI:SS',\r      MODEL_TYPE VARCHAR(800) CHARACTER SET UNICODE NOT CASESPECIFIC,\r      TARGET FLOAT,\r      CREATE_DT_DAY DATE FORMAT 'YYYY-MM-DD')\rPRIMARY INDEX ( ROW_ID )\rPARTITION BY RANGE_N(CREATE_DT_DAY  BETWEEN DATE '2020-01-01' AND DATE '2025-01-01' EACH INTERVAL '1' DAY );",

In [14]:
my_sql = '''SELECT TOP 1 
                 *    
             FROM PRD_DB_CLIENT4D_DEV1.{}
             --WHERE CREATE_DT_DAY = '2021-10-18'
         '''.format(table_name)

curs.execute(my_sql)
cols = [desc[0] for desc in curs.description]

In [15]:
my_sql = '''SELECT TOP 1000 
                 *    
             FROM PRD_DB_CLIENT4D_DEV1.{}
             --WHERE CREATE_DT_DAY = '2021-10-18'
         '''.format(table_name)
curs.execute(my_sql)

res = pd.DataFrame(curs.fetchall(), columns=cols)

res.head(20)

,ROW_ID,INN,MODEL_NAME,HYPOTHESIS,CLIENT_TYPE,ID_SCEN,CREATE_DT,ONLY_SB,DEAL_EVENT_DT,MODEL_TYPE,TARGET,CREATE_DT_DAY
0,1113,7810245481,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
1,7889,261005469112,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
2,9721,280118502932,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
3,9862,500305708289,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
4,1565,6664013640,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
5,993,770802303187,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
6,1687,772740007083,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
7,3525,7714928492,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
8,4358,230302628903,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18
9,1914,343659106939,1-290ZX5NB,validation,ksb,test35,2021-10-18 13:54:44,0.0,2021-10-18 13:54:44,lal,1.0,2021-10-18


In [16]:
curs.close()

## Using PySpark backened with jDBC driver

In [33]:
my_sql = '''(SELECT TOP 100 
                 *
             FROM {db}.{tbl}) as t
         '''.format(db=DB, tbl = table_name )

sdf = sp.get_teradata(db, my_sql)

In [35]:
sdf.dtypes

[('ROW_ID', 'int'),
 ('INN', 'string'),
 ('MODEL_NAME', 'string'),
 ('HYPOTHESIS', 'string'),
 ('CLIENT_TYPE', 'string'),
 ('ID_SCEN', 'string'),
 ('CREATE_DT', 'timestamp'),
 ('ONLY_SB', 'float'),
 ('DEAL_EVENT_DT', 'timestamp'),
 ('MODEL_TYPE', 'string'),
 ('TARGET', 'float'),
 ('CREATE_DT_DAY', 'date')]

In [34]:
sdf.show()

,ROW_ID,INN,MODEL_NAME,HYPOTHESIS,CLIENT_TYPE,ID_SCEN,CREATE_DT,ONLY_SB,DEAL_EVENT_DT,MODEL_TYPE,TARGET,CREATE_DT_DAY
0,78692,861204106102,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
1,139113,3249003661,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
2,229857,7325101364,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
3,64050,6317104240,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
4,153978,0256014475,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
5,131302,164491236155,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
6,41353,5834115946,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
7,125510,3241501190,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
8,10948,5404342739,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
9,222555,7716805016,1-115HHZO,original dataset,all,test38,2021-11-19 13:14:33,0.0,2021-11-19 13:14:33,lal,1.0,2021-11-19
